# Graphing Relationships with Neo4j
--------------------------------------------------------------------------

This tutorial makes use of the _Musicbrainz_ database: [https://musicbrainz.org/](https://musicbrainz.org/)

In [1]:
#!pip install py2neo
#!pip install neo4j-driver===1.6.2
#!pip install pymongo

In [2]:
from neo4j import *
from py2neo import *
from pymongo import MongoClient

In [3]:
# http://localhost:7474
# https://neo4j.com/docs/

In [5]:
#db = Database("http://cc.unmrds.net:7474/db/data/")
db = Database("http://localhost:7474/db/data/", user="neo4j", password='neo4j2')

In [6]:
from py2neo import Graph, Path
#graph = Graph("http://cc.unmrds.net:7474/db/data/")
graph = Graph("http://localhost:7474/db/data/", user="neo4j", password='neo4j2')

In [8]:
#client = MongoClient('mongo', 27017)
client = MongoClient('localhost', 27017)

In [9]:
db = client.jazz_catalog

In [10]:
db.artists.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1504

In [11]:
# use MERGE for unique

#for artist in db.artists.find():
#    graph.run("MERGE (artist:Artist {name:{name}, id:{id}}) RETURN artist", name=artist['name'], id=artist['id'])

In [12]:
c = graph.run("MATCH (n:Artist) WHERE n.name = 'Miles Davis' RETURN n.id")

In [13]:
for i in c:
    print(i)

<Record n.id='561d854a-6a28-4aa7-8c99-323e6ce46c2a'>


```
# set instruments for artists
# create node for instruments and add 'plays' relationship

run_merge = "MERGE (instrument:Instrument {name: {name}}) RETURN instrument.name"
run_match = "MATCH (a:Artist), (b:Instrument) WHERE a.id = {aid} AND b.name = {name} MERGE (a)-[r:PLAYS {name: a.name + '->' + b.name}]->(b)"

for r in db.releases.find():
    if 'medium-list' in r:
        media = r['medium-list']
        for m in media:
            if 'track-list' in m:
                tracks = m['track-list']
                for each in tracks:
                    if 'recording' in each.keys():
                        if 'artist-relation-list' in each['recording'].keys():
                            artist_list = each['recording']['artist-relation-list']
                            for a in artist_list:
                                artist_id = a['artist']['id']
                                if 'attribute-list' in a.keys():
                                    instruments = a['attribute-list']
                                    for inst in instruments:
                                        graph.run(run_merge, name=inst)
                                        graph.run(run_match, aid=artist_id, name=inst)
```

## CRUD Operations

The basic operations can be demonstrated in the Neo4j browser: [http://localhost:7474/browser/](http://localhost:7474/browser/).

**Create**

Neo4j's _Cypher_ query language provides a CREATE clause, but we use the MERGE statement to enforce uniqueness.

```
CREATE (artist:Artist {name: 'Billie Holiday', id: 'd59c4cda-11d9-48db-8bfe-b557ee602aed'}) RETURN artist

MERGE (artist:Artist {name: 'Billie Holiday', id: 'd59c4cda-11d9-48db-8bfe-b557ee602aed'}) RETURN artist

MERGE (instrument:Instrument {name: 'voice'}) RETURN instrument

MATCH (a:Artist), (b:Instrument) WHERE a.name = 'Billie Holiday' AND b.name = 'voice' MERGE (a)-[r:PLAYS {name: a.name + '->' + b.name}]->(b)"
```

**Retrieve**

```
MATCH (n) RETURN n LIMIT 100

MATCH (n:Instrument) RETURN n LIMIT 100

MATCH (:Artist { name: 'Billie Holiday' })--(instrument) RETURN instrument.name

MATCH (:Artist { name: 'Ron Carter' })--(instrument) RETURN instrument.name
```

That last query in pseudo-SQL:

```
SELECT artist.name, instrument.name
FROM artist
INNER JOIN instrument ON artist.id = instrument.artist_id
WHERE artist.name == 'Ron Carter'
```

Retrieve relationships:

```
MATCH (:Artist { name: 'Billie Holiday' })-[r]->(instrument) RETURN type(r)

MATCH (:Artist { name: 'Ron Carter' })-[r]->(instrument) RETURN type(r)

MATCH p=(:Artist { name: 'Billie Holiday' })-[r:PLAYS]->() RETURN p

MATCH p=(:Artist { name: 'Ron Carter' })-[r:PLAYS]->() RETURN p

MATCH p=()-[r:PLAYS]->(:Instrument { name: 'bass' }) RETURN p LIMIT 50
```

**Update**

Add a property:

```
MATCH (n:Artist { name: 'Miles Davis' }) SET n.dbseed = TRUE RETURN n.name, n.dbseed

then try

MATCH (n:Artist { name: 'Miles Davis' }) RETURN n
```

Remove a property:

```
MATCH (n:Artist { name: 'Miles Davis' }) SET n.dbseed = NULL RETURN n.name
```

Note that there is also a REMOVE clause that provides similar functionality.

**Delete**

Deletes nodes, relationships, or paths.

```
MATCH (n:Artist { name: 'Romeo Penque' }) DELETE n

MATCH (n:Artist { name: 'Romeo Penque' }) DETACH DELETE n

MATCH (n:Artist { name: 'Frank Rehak' })-[r:PLAYS]->() DELETE r

MATCH (n:Artist { name: 'Frank Rehak' }) DELETE n
```

**Remove**

Removes properties and labels.

```
MATCH (n:Artist { name: 'Miles Davis' }) SET n.dbseed = TRUE RETURN n.name, n.dbseed

MATCH (n:Artist { name: 'Miles Davis' }) REMOVE n.dbseed RETURN n.name
```

In [14]:
rt = db.releases.find_one()

In [15]:
rt['release-event-list'][0]['date']

'1954'

```
# created 'played_for' and 'played_with' relationships
# for each release
#    - store credited_artist_id (c)
#    - store artist_rel_ids (a)
# if c == a then pass
# for every a: a played_for -> c and a played_with -> other a
'''
play_for = "MATCH (a:Artist), (b:Artist) WHERE a.id = {aid} AND b.id = {bid} MERGE (a)-[r:PLAYED_FOR {name: a.name + '<->' + b.name, date: {rdate}}]->(b)"
play_with = "MATCH (a:Artist), (b:Artist) WHERE a.id = {aid} AND b.id = {bid} MERGE (a)-[r:PLAYED_WITH {name: a.name + '<->' + b.name, date: {rdate}}]->(b)"

count = 1

for r in db.releases.find():
    try:
        bandmate_ids = []
        bandleader = ''
        release_year = ''
        if 'artist-credit' in r:
            bandleader = r['artist-credit'][0]['artist']['id']
        if 'release-event-list' in r:
            release_year = r['release-event-list'][0]['date']
        if 'medium-list' in r:
            media = r['medium-list']
            for m in media:
                if 'track-list' in m:
                    tracks = m['track-list']
                    for each in tracks:
                        if 'recording' in each.keys():
                            if 'artist-relation-list' in each['recording'].keys():
                                artist_list = each['recording']['artist-relation-list']
                                for a in artist_list:
                                    if a['artist']['id'] == bandleader:
                                        pass
                                    else:
                                        bandmate_ids.append(a['artist']['id'])
        for musician in bandmate_ids:
            graph.run(play_for, aid = musician, bid = bandleader, rdate = release_year)
            for bandmates in bandmate_ids:
                if bandmates == musician:
                    pass
                else:
                    graph.run(play_with, aid = musician, bid = bandmates, rdate= release_year)
    except Exception as e:
        print(e)
        pass
'''
```

## Querying Paths

Query relationships:

```
MATCH (a:Artist)-[:PLAYED_FOR_WHEN]-(bandleaders)
WHERE a.name = "Wayne Shorter" 
RETURN a, bandleaders

MATCH (a:Artist)-[:PLAYED_WITH]-(bandmates)
WHERE a.name = "Wayne Shorter" 
RETURN a, bandmates

MATCH (a:Artist)-[:PLAYED_FOR]->(bandleader1)-[:PLAYED_FOR]->(bandleader2)
RETURN a.name, bandleader1.name, bandleader2.name

MATCH (a:Artist)-[:PLAYED_WITH*..2]->(player)-[:PLAYS]->(i:Instrument)
WHERE a.name = 'Miles Davis' AND i.name = 'piano'
RETURN player.name
```

Query paths

```
MATCH p=(a:Artist)-[r:PLAYED_FOR_WHEN]->(bandleader1)-[s:PLAYED_FOR_WHEN]->(bandleader2)
WHERE s.date = '1963'
RETURN p LIMIT 5

MATCH p=(a:Artist)-[r:PLAYED_FOR*..2]->()
WHERE a.name = 'Herbie Hancock'
RETURN p

MATCH p=(a:Artist)-[r:PLAYED_FOR*..2]->(bandleader)
WHERE a.name = 'Herbie Hancock'
RETURN bandleader.name, p

MATCH p=(a:Artist)-[:PLAYED_WITH*..2]->(player)-[:PLAYS]->(i:Instrument)
WHERE a.name = 'Miles Davis' AND i.name = 'piano'
RETURN p

MATCH p=(a:Artist)-[:PLAYED_WITH*..2]->(player)-[:PLAYS]->(i:Instrument)
WHERE a.name = 'Miles Davis' AND i.name = 'piano'
RETURN p

MATCH p=(a:Artist)-[:PLAYED_WITH*..2]->(b:Artist)-[:PLAYS]->(i:Instrument)
WHERE a.name = 'Lee Konitz' AND b.name = 'Hank Jones' AND i.name = 'piano'
RETURN p

MATCH p = shortestPath((a:Artist)-[:PLAYED_WITH*..5]-(b:Artist))
WHERE a.name = 'Lee Konitz' AND b.name = 'Hank Jones'
RETURN p

MATCH (a:Artist { name: 'Lee Konitz' }),(i:Instrument { name: 'piano' }), p = allShortestPaths((a)-[*]-(i))
RETURN p

"Friend of a friend"

MATCH (a:Artist { name: 'Lee Konitz' }),(b:Artist{name: "Herbie Hancock"}), p = shortestPath((a)-[*]-(b))
RETURN p

MATCH (a:Artist { name: 'Lee Konitz' }),(b:Artist{name: "Herbie Hancock"}), p = shortestPath((a)-[*]->(b))
RETURN p
```